### data preprocess 

In [1]:
import numpy as np
import tensorflow as tf

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

In [2]:
dataset_path = os.listdir('./dataset')


In [3]:
print (dataset_path)  #what kinds of classes are in this dataset

print("Types of classes labels found: ", len(dataset_path))

['bolero', 'danca', 'pop']
Types of classes labels found:  3


In [4]:
class_labels = []

for item in dataset_path:
 # Get all the file names
 all_classes = os.listdir('./dataset' + '/' +item)
 #print(all_classes)

 # Add them to the list
 for room in all_classes:
    class_labels.append((item, str('dataset_path' + '/' +item) + '/' + room))
    #print(class_labels[:5])

In [5]:
# Build a dataframe        
df = pd.DataFrame(data=class_labels, columns=['Labels', 'image'])
print(df.head())
print(df.tail())

   Labels                                              image
0  bolero  dataset_path/bolero/Ai_Khổ_Vì_Ai_BeeBoss_part1...
1  bolero  dataset_path/bolero/Ai_Khổ_Vì_Ai_BeeBoss_part1...
2  bolero  dataset_path/bolero/Ai_Khổ_Vì_Ai_BeeBoss_part2...
3  bolero  dataset_path/bolero/Ai_Khổ_Vì_Ai_BeeBoss_part2...
4  bolero  dataset_path/bolero/Ai_Khổ_Vì_Ai_BeeBoss_part3...
     Labels                                              image
2995    pop          dataset_path/pop/Ơ_Thế_Là_Kim_Chi_Sun.png
2996    pop  dataset_path/pop/Ấn_Nút_Nhớ..._Thả_Giấc_Mơ_Sơn...
2997    pop  dataset_path/pop/Ấn_Nút_Nhớ..._Thả_Giấc_Mơ_Sơn...
2998    pop  dataset_path/pop/Ấn_Nút_Nhớ..._Thả_Giấc_Mơ_Sơn...
2999    pop  dataset_path/pop/Ấn_Nút_Nhớ..._Thả_Giấc_Mơ_Sơn...


In [6]:
# Let's check how many samples for each category are present
print("Total number of images in the dataset: ", len(df))

label_count = df['Labels'].value_counts()
print(label_count)

Total number of images in the dataset:  3000
Labels
bolero    1000
danca     1000
pop       1000
Name: count, dtype: int64


In [7]:
from PIL import Image
import numpy as np
import os

path = './dataset/'
dataset_path = os.listdir(path)

im_size = 224

images = []
labels = []

for label in dataset_path:
    data_path = os.path.join(path, label)
    filenames = os.listdir(data_path)

    for f in filenames:
        img_path = os.path.join(data_path, f)

        try:
            img = Image.open(img_path).convert("RGB")
            img = img.resize((im_size, im_size))
            img = np.array(img)  # chuyển thành numpy array giống cv2
            images.append(img)
            labels.append(label)
        except Exception as e:
            print(f"❌ Lỗi đọc ảnh {img_path}: {e}")


In [8]:
images = np.array(images)

images = images.astype('float32') / 255.0   
images.shape

(3000, 224, 224, 3)

In [9]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
y=df['Labels'].values
print(y)

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
print (y)

['bolero' 'bolero' 'bolero' ... 'pop' 'pop' 'pop']
[0 0 0 ... 2 2 2]


In [10]:
y=y.reshape(-1,1)

from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(y) #.toarray()
print(Y[:5])
print(Y[35:])

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


# images, Y = shuffle(images, Y, random_state=1)


# train_x, valid_x, train_y, valid_y = train_test_split(images, Y, test_size=0.05, random_state=415)

# #inpect the shape of the training and testing.
# print(train_x.shape)
# print(train_y.shape)
# print(valid_x.shape)
# print(valid_y.shape)


# Shuffle toàn bộ
images, Y = shuffle(images, Y, random_state=1)

# Lần 1: chia train + temp (95%)
train_x, temp_x, train_y, temp_y = train_test_split(images, Y, test_size=0.25, random_state=415)

# Lần 2: chia temp thành val + test (7.5% + 7.5%)
val_x, test_x, val_y, test_y = train_test_split(temp_x, temp_y, test_size=0.4, random_state=415)
#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

In [31]:
import numpy as np

# Chuyển từ one-hot về label index
val_labels = np.argmax(train_y, axis=1)
test_labels = np.argmax(test_y, axis=1)

# Đếm số lượng mỗi class
print("Phân bố tập validation:", np.bincount(val_labels))
print("Phân bố tập test:", np.bincount(test_labels))


Phân bố tập validation: [747 750 753]
Phân bố tập test: [ 94 109  97]


In [32]:
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0

NUM_CLASSES = 3
IMG_SIZE = 224
size = (IMG_SIZE, IMG_SIZE)


inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))


# Using model without transfer learning

outputs = EfficientNetB0(include_top=True, weights=None, classes=NUM_CLASSES)(inputs)

In [36]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(
    filepath='./check_point/best_model_epoch_{epoch:02d}_acc_{val_accuracy:.2f}.h5',      # Tên file lưu model tốt nhất
    monitor='val_accuracy',        # Chọn tiêu chí để đánh giá "tốt nhất"
    save_best_only=True,           # Chỉ lưu khi tốt hơn trước
    mode='max',                    # Nếu dùng 'val_accuracy' => 'max'; nếu 'val_loss' => 'min'
    verbose=2
)

model = tf.keras.Model(inputs, outputs)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"] )

model.summary()

hist = model.fit(train_x, train_y,validation_data=(val_x, val_y), epochs=30, verbose=2,callbacks=[checkpoint])

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 3)              │     4,053,414 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,053,414 (15.46 MB)

 Trainable params: 4,011,391 (15.30 MB)

 Non-trainable params: 42,023 (164.16 KB)

...

71/71 - 66s - 930ms/step - accuracy: 0.9302 - loss: 0.2176 - val_accuracy: 0.5711 - val_loss: 3.5323
Epoch 4/30

Epoch 4: val_accuracy did not improve from 0.57111
71/71 - 66s - 931ms/step - accuracy: 0.9382 - loss: 0.1835 - val_accuracy: 0.5511 - val_loss: 1.7974
Epoch 5/30

Epoch 5: val_accuracy did not improve from 0.57111
71/71 - 65s - 921ms/step - accuracy: 0.8862 - loss: 0.3178 - val_accuracy: 0.4244 - val_loss: 3.1961
Epoch 6/30

Epoch 6: val_accuracy improved from 0.57111 to 0.63778, saving model to ./check_point/best_model_epoch_06_acc_0.64.h5


71/71 - 67s - 944ms/step - accuracy: 0.9564 - loss: 0.1189 - val_accuracy: 0.6378 - val_loss: 1.5552
Epoch 7/30

Epoch 7: val_accuracy did not improve from 0.63778
71/71 - 65s - 921ms/step - accuracy: 0.9676 - loss: 0.0850 - val_accuracy: 0.5978 - val_loss: 1.6948
Epoch 8/30

Epoch 8: val_accuracy improved from 0.63778 to 0.72667, saving model to ./check_point/best_model_epoch_08_acc_0.73.h5


71/71 - 67s - 939ms/step - accuracy: 0.9711 - loss: 0.0939 - val_accuracy: 0.7267 - val_loss: 1.1027
Epoch 9/30

Epoch 9: val_accuracy did not improve from 0.72667
71/71 - 65s - 917ms/step - accuracy: 0.9747 - loss: 0.0675 - val_accuracy: 0.6711 - val_loss: 1.5800
Epoch 10/30

Epoch 10: val_accuracy did not improve from 0.72667
71/71 - 66s - 931ms/step - accuracy: 0.9773 - loss: 0.0643 - val_accuracy: 0.6889 - val_loss: 1.7657
Epoch 11/30

Epoch 11: val_accuracy did not improve from 0.72667
71/71 - 65s - 909ms/step - accuracy: 0.9733 - loss: 0.0727 - val_accuracy: 0.6244 - val_loss: 1.7652
Epoch 12/30

Epoch 12: val_accuracy did not improve from 0.72667
71/71 - 65s - 923ms/step - accuracy: 0.9738 - loss: 0.0753 - val_accuracy: 0.5756 - val_loss: 2.6184
Epoch 13/30

Epoch 13: val_accuracy did not improve from 0.72667
71/71 - 65s - 913ms/step - accuracy: 0.9787 - loss: 0.0674 - val_accuracy: 0.6600 - val_loss: 1.5940
Epoch 14/30

Epoch 14: val_accuracy did not improve from 0.72667
71/71 

71/71 - 66s - 934ms/step - accuracy: 0.9836 - loss: 0.0493 - val_accuracy: 0.7289 - val_loss: 1.4129
Epoch 23/30

Epoch 23: val_accuracy improved from 0.72889 to 0.73778, saving model to ./check_point/best_model_epoch_23_acc_0.74.h5


71/71 - 65s - 922ms/step - accuracy: 0.9884 - loss: 0.0553 - val_accuracy: 0.7378 - val_loss: 0.9819
Epoch 24/30

Epoch 24: val_accuracy did not improve from 0.73778
71/71 - 65s - 922ms/step - accuracy: 0.9644 - loss: 0.1324 - val_accuracy: 0.6467 - val_loss: 1.9521
Epoch 25/30

Epoch 25: val_accuracy did not improve from 0.73778
71/71 - 65s - 912ms/step - accuracy: 0.9560 - loss: 0.1606 - val_accuracy: 0.5133 - val_loss: 1.9967
Epoch 26/30

Epoch 26: val_accuracy did not improve from 0.73778
71/71 - 66s - 923ms/step - accuracy: 0.8742 - loss: 0.3638 - val_accuracy: 0.5556 - val_loss: 1.5659
Epoch 27/30

Epoch 27: val_accuracy did not improve from 0.73778
71/71 - 65s - 914ms/step - accuracy: 0.9547 - loss: 0.1225 - val_accuracy: 0.4600 - val_loss: 3.7296
Epoch 28/30

Epoch 28: val_accuracy did not improve from 0.73778
71/71 - 66s - 924ms/step - accuracy: 0.9760 - loss: 0.0631 - val_accuracy: 0.6978 - val_loss: 1.3019
Epoch 29/30

Epoch 29: val_accuracy did not improve from 0.73778
71/7

In [37]:
# Lưu toàn bộ mô hình (kiến trúc + trọng số)
model.save("./model/efficientnet_model_best.keras")


In [51]:
from tensorflow.keras.models import load_model

efficientnet_model = load_model("./check_point/check_point\best_model_epoch_23_acc_0.74.h5")

OSError: [Errno 22] Unable to synchronously open file (unable to open file: name = './check_point/check_pointest_model_epoch_23_acc_0.74.h5', errno = 22, error message = 'Invalid argument', flags = 0, o_flags = 0)

In [42]:
### test model 
from tensorflow.keras.models import load_model

# Load mô hình tốt nhất
best_model = load_model('./model/efficientnet_model_best.keras')

# Đánh giá mô hình tốt nhất trên tập test
test_loss, test_accuracy = best_model.evaluate(test_x, test_y, verbose=2)

print(f"✅ Best Model Test Accuracy: {test_accuracy:.4f}")
print(f"📉 Best Model Test Loss: {test_loss:.4f}")


d:\classified_music\.venv\lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 215 variables whereas the saved optimizer has 428 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


10/10 - 5s - 480ms/step - accuracy: 0.7367 - loss: 1.0615
✅ Best Model Test Accuracy: 0.7367
📉 Best Model Test Loss: 1.0615


In [43]:
# Convert sang TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(efficientnet_model)
tflite_model = converter.convert()

# Lưu ra file
with open("./model/efficientnet_model.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\BHM-KY~1\AppData\Local\Temp\2\tmptpj_47nj\assets


INFO:tensorflow:Assets written to: C:\Users\BHM-KY~1\AppData\Local\Temp\2\tmptpj_47nj\assets


Saved artifact at 'C:\Users\BHM-KY~1\AppData\Local\Temp\2\tmptpj_47nj'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_8')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  1745357523600: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  1749632012480: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  1749751564416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1749751569168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1749751570928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1749751567936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1749751569696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1749751576384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1749751574448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1749751578320: TensorSpec(shape=(), dtype=tf.reso

In [44]:
from tensorflow.keras.models import load_model

interpreter = tf.lite.Interpreter(model_path="./model/efficientnet_model.tflite")
interpreter.allocate_tensors()

d:\classified_music\.venv\lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [ ]:
import numpy as np
from PIL import Image

def preprocess_image(image_path, img_size=224):
    img = Image.open(image_path).resize((img_size, img_size))
    img = img.convert("RGB")
    img = np.array(img, dtype=np.float32) / 255.0  # normalize về [0,1]
    img = np.expand_dims(img, axis=0)  # (1, 224, 224, 3)
    return img

In [48]:

import tensorflow as tf
import numpy as np
from PIL import Image

# ======= Cấu hình ============
MODEL_PATH = "./model/efficientnet_model.tflite"
IMAGE_PATH = r"D:\classified_music\dataset\bolero\Bông_Ô_Môi_BeeBoss_part2_part1.png"
IMG_SIZE = 224
NUM_CLASSES = 3  # sửa lại theo mô hình bạn

# ======= Tiền xử lý ảnh ============
def preprocess_image(image_path, img_size):
    img = Image.open(image_path).convert("RGB")
    img = img.resize((img_size, img_size))
    img_array = np.array(img, dtype=np.float32) / 255.0  # normalize về [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # thêm batch dimension: (1, 224, 224, 3)
    return img_array

# ======= Load model ============
interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# ======= Load và chạy ảnh test ============
input_data = preprocess_image(IMAGE_PATH, IMG_SIZE)

# Đưa ảnh vào model
interpreter.set_tensor(input_details[0]['index'], input_data)

# Chạy model
interpreter.invoke()

# Lấy kết quả đầu ra
output_data = interpreter.get_tensor(output_details[0]['index'])  # shape: (1, NUM_CLASSES)
predicted_class = np.argmax(output_data)
confidence = np.max(output_data)

print("🧠 Dự đoán lớp:", predicted_class)
print("✅ Độ tin cậy:", round(confidence * 100, 2), "%")
# bolero : 0 , dan ca:1, pop:2

🧠 Dự đoán lớp: 0
✅ Độ tin cậy: 94.69 %


In [1]:
import zipfile
with zipfile.ZipFile(r"D:\classified_music\dowload.zip", 'r') as zip_ref:
    zip_ref.extractall(r"D:\classified_music\dowload")

In [1]:
import os
import shutil
import random

# Cấu hình
input_dir = r"D:\classified_music\data"  # thư mục gốc chứa các class
output_dir = r"D:\classified_music\dataset"  # thư mục lưu train/val/test
train_ratio = 0.6
val_ratio = 0.25
test_ratio = 0.15

assert train_ratio + val_ratio + test_ratio == 1.0, "Tổng tỉ lệ phải = 1.0"

# Tạo thư mục output
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(output_dir, split), exist_ok=True)

# Duyệt qua từng class
for class_name in os.listdir(input_dir):
    class_path = os.path.join(input_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    # Danh sách ảnh và shuffle ngẫu nhiên
    images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
    random.shuffle(images)

    total = len(images)
    train_count = int(total * train_ratio)
    val_count = int(total * val_ratio)
    test_count = total - train_count - val_count  # còn lại

    split_data = {
        'train': images[:train_count],
        'val': images[train_count:train_count + val_count],
        'test': images[train_count + val_count:]
    }

    # Copy từng ảnh sang thư mục tương ứng
    for split in ['train', 'val', 'test']:
        split_dir = os.path.join(output_dir, split, class_name)
        os.makedirs(split_dir, exist_ok=True)
        for img_name in split_data[split]:
            src_path = os.path.join(class_path, img_name)
            dst_path = os.path.join(split_dir, img_name)
            shutil.copyfile(src_path, dst_path)

print("✅ Dữ liệu đã được chia đều và random.")


✅ Dữ liệu đã được chia đều và random.
